# Post-hoc statistics about generated results

## Imports

In [1]:
import os


from paths import word_plot_res_path
from reshandler import ResHandler
from measurements import *

## 1. Direction Consistency

In [3]:
vars = []
reshandler = ResHandler()

for datapath in os.listdir(word_plot_res_path): 
    reshandler.path = os.path.join(word_plot_res_path, datapath)
    reshandler.read()
    res = reshandler.data

    var = DirectionalConsistency.analyze(res)
    vars.append(var)

In [6]:
sum(vars)/len(vars)

0.6239262277909413